In [1]:
import os
import json
import re
from datetime import datetime
from pathlib import Path
from shutil import copyfile
from urllib.parse import quote_plus

import nltk
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from kaggle_secrets import UserSecretsClient
!pip install --quiet langchain langchain-community unstructured[docx]

from sentence_transformers import SentenceTransformer
from transformers import pipeline
# from langchain_unstructured import UnstructuredLoader as UnstructuredFileLoader
from langchain_community.document_loaders import UnstructuredFileLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.6/212.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-

2025-07-22 17:17:16.907394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753204637.145032      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753204637.213276      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# === NLTK Setup ===
required_nltk_packages = [
    "punkt", "averaged_perceptron_tagger", "maxent_ne_chunker", "words", "stopwords"
]

def nltk_package_path(package):
    return {
        "punkt": "tokenizers/punkt",
        "averaged_perceptron_tagger": "taggers/averaged_perceptron_tagger",
        "maxent_ne_chunker": "chunkers/maxent_ne_chunker",
        "words": "corpora/words",
        "stopwords": "corpora/stopwords"
    }.get(package, package)

for package in required_nltk_packages:
    try:
        nltk.data.find(f"{nltk_package_path(package)}")
    except LookupError:
        nltk.download(package)

# === Models ===
embedding_model = SentenceTransformer("BAAI/bge-small-en-v1.5")
llm = pipeline("text2text-generation", model="google/flan-t5-large", device=0, max_new_tokens=512)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [3]:
# === Config ===
# WATCH_FOLDER_ID = "17sSWn0cfX-jEFmgG4Hl-P9_NtG9kqS6D"
WATCH_FOLDER_ID="1tJhw1KbempeSjoHfkJ-Ig6Ikn-bKL20Q" #test folder
DOWNLOAD_PATH = Path("/tmp/gdrive_docs")
DOWNLOAD_PATH.mkdir(parents=True, exist_ok=True)

# === Auth ===
CRED_INPUT = "/kaggle/input/google-cred/credentials.json"
CRED_WORKING = "/kaggle/working/credentials.json"
copyfile(CRED_INPUT, CRED_WORKING)

gauth = GoogleAuth()
gauth.LoadCredentialsFile(CRED_WORKING)
if gauth.credentials is None:
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    gauth.Refresh()
else:
    gauth.Authorize()
gauth.SaveCredentialsFile(CRED_WORKING)
drive = GoogleDrive(gauth)


In [4]:
# === HELPERS ===
def sanitize_filename(title):
    return re.sub(r'[\\/*?:"<>|]', "_", title)

def detect_format_style(text):
    bold_qs = len(re.findall(r"\n[A-Z][^\n]{4,100}\?\s*\n", text))
    return "bolded_qs" if bold_qs > 3 else "narrative"

def llm_extract_qa_pairs(text):
    prompt = (
        """Extract question and answer pairs from the grant text below. Use format:
Q: [Question]
A: [Answer]

Text:
"""
        f"{text}"
    )
    try:
        output = llm(prompt.strip())[0]["generated_text"]
        pairs = re.findall(r"Q:\s*(.+?)\s*A:\s*(.+?)(?=\nQ:|\Z)", output, re.DOTALL)
        return [{"question": q.strip(), "answer": a.strip()} for q, a in pairs]
    except Exception as e:
        print(f"⚠️ LLM Q&A extraction failed: {e}")
        return []

def extract_qa_pairs(text):
    q_pattern = re.compile(r"(?=(?:^|\n)([^:\n]{4,100}[\?:])\s*\n?)", re.MULTILINE)
    splits = q_pattern.split(text)
    qa_pairs = []
    for i in range(1, len(splits), 2):
        question = splits[i].strip()
        answer = splits[i+1].strip() if i+1 < len(splits) else ""
        if len(answer) > 10 and len(question) > 5:
            qa_pairs.append({"question": question, "answer": answer})
    return qa_pairs

def categorize_chunk(text: str, question: str = "") -> list:
    categories = []
    lower_text = text.lower()
    lower_q = question.lower()
    if "problem" in lower_q or "we address" in lower_text:
        categories.append("Contact Information Problem")
    if "mission" in lower_q or "mission" in lower_text:
        categories.append("Mission Statement")
    if len(text) < 400:
        categories.append("Project Summary")
    if "goal" in lower_q or "vision" in lower_q:
        categories.append("Goals, Vision, or Objectives")
    if "approach" in lower_q or "methodology" in lower_text:
        categories.append("Our Solution or Approach")
    if "impact" in lower_q or "results" in lower_q:
        categories.append("Impact Results or Outcomes")
    if "who benefits" in lower_q or "target population" in lower_q:
        categories.append("Beneficiaries")
    if "unique" in lower_text:
        categories.append("Unique Value Proposition")
    if "timeline" in lower_q:
        categories.append("Plan and Timeline")
    if "budget" in lower_q or "$" in text:
        categories.append("Budget and Funding")
    if "sustainability" in lower_text:
        categories.append("Sustainability or Strategy")
    if "team" in lower_q or "lived experience" in lower_text:
        categories.append("Team Members and Descriptions")
    if re.search(r"\\bfounded\\b|\\bhistory\\b|\\baccelerator\\b", lower_text):
        categories.append("Organizational History")
    if re.search(r"(https?://\\S+)", lower_text):
        categories.append("Supplementary Materials")
    if not categories:
        categories.append("Miscellaneous")
    return categories

def annotate_chunk(chunk_text: str) -> dict:
    prompt = (
        """You are a grant document assistant.
Analyze the following text and return the most likely category it fits into.
Possible categories include: Mission Statement, Problem, Team, Impact, Sustainability, etc.
Return only JSON in the format:
{"question": "...", "answer": "...", "category": "..."}

TEXT:
"""
        f"{chunk_text}"
    )
    try:
        result = llm(prompt.strip())[0]['generated_text']
        match = re.search(r"\{.*\}", result, re.DOTALL)
        if match:
            return json.loads(match.group())
        else:
            raise ValueError("No JSON found in model output")
    except Exception as e:
        print(f"⚠️ LLM error: {e}")
        return {"question": None, "answer": None, "category": "Miscellaneous"}

In [5]:
# === FILE CHECKER ===
def check_for_new_files():
    print(f"\U0001F50D Checking for new files at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    file_list = drive.ListFile({'q': f"'{WATCH_FOLDER_ID}' in parents and trashed=false"}).GetList()

    for file in file_list:
        title = file['title']
        file_id = file['id']
        modified_time = file['modifiedDate']
        sanitized_title = sanitize_filename(title)
        file_path = DOWNLOAD_PATH / f"{sanitized_title}.docx"

        # existing = structured_col.find_one({"metadata.title": title})
        # if existing and existing.get("metadata", {}).get("modifiedDate") == modified_time:
        #     print(f"✅ Already processed: {title}")
        #     continue

        print(f"⬇ Downloading: {title}")
        file.GetContentFile(str(file_path), mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document')

        loader = UnstructuredFileLoader(str(file_path))
        pages = loader.load()
        raw_text = pages[0].page_content
        doc_id = title.replace(" ", "_").lower()

        qa_pairs = extract_qa_pairs(raw_text)

        doc_entry = {
            "doc_id": doc_id,
            "metadata": {
                "title": title,
                "doc_id": doc_id,
                "file_id": file_id,
                "modifiedDate": modified_time,
                "word_count": len(raw_text.split()),
            },
            "chunks": []
        }

        flat_chunks = []

        for i, pair in enumerate(qa_pairs):
            q, a = pair['question'], pair['answer']
            categories = categorize_chunk(a, q)
            if categories == ["Miscellaneous"]:
                annotations = annotate_chunk(a)
                categories = [annotations["category"]] if annotations["category"] else ["Miscellaneous"]
            for cat in categories:
                chunk_id = f"{doc_id}_chunk_{i}_{cat.replace(' ', '_')}"
                try:
                    embedding = embedding_model.encode(a).tolist()
                except Exception as e:
                    print(f"⚠️ Embedding error: {e}")
                    embedding = None

                chunk_data = {
                    "chunk_id": chunk_id,
                    "text": a,
                    "embedding": embedding,
                    "question": q,
                    "answer": a,
                    "word_count": len(a.split()),
                    "metadata": {
                        "doc_id": doc_id,
                        "title": title,
                        "file_id": file_id,
                        "modifiedDate": modified_time,
                        "category": cat
                    }
                }
                doc_entry["chunks"].append(chunk_data)
                flat_chunks.append(chunk_data)

        # structured_col.delete_many({"metadata.title": title})
        # flat_col.delete_many({"metadata.doc_id": doc_id})

        # structured_col.insert_one(doc_entry)
        # if flat_chunks:
        #     flat_col.insert_many(flat_chunks)
        with open(f"/kaggle/working/{doc_id}_structured.json", "w") as f:
            json.dump(doc_entry, f, indent=2)
        with open(f"/kaggle/working/{doc_id}_flat.json", "w") as f:
            json.dump(flat_chunks, f, indent=2)


        print(f"✅ Processed and inserted: {title}")

        try:
            os.remove(file_path)
        except Exception as e:
            print(f"⚠️ Failed to delete file: {e}")

    print("✅ Check complete.\n")

if __name__ == "__main__":
    check_for_new_files()
# === LOOP ===
# while True:
#     check_for_new_files()
#     print("⏲️ Sleeping for 5 minutes...\n")
#     time.sleep(300)

🔍 Checking for new files at 2025-07-22 17:18:11
⬇ Downloading: Copy of DATA - Queens Tech Challenge


/tmp/ipykernel_36/444122703.py:21: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader(str(file_path))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Processed and inserted: Copy of DATA - Application YASS
✅ Check complete.



In [6]:
# # === Main ===
# def check_for_new_files():
#     print(f"🔍 Checking for new files at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
#     file_list = drive.ListFile({'q': f"'{WATCH_FOLDER_ID}' in parents and trashed=false"}).GetList()

#     for file in file_list:
#         title = file['title']
#         file_id = file['id']
#         modified_time = file['modifiedDate']

#         # Make filename safe
#         doc_id = sanitize_filename(title.replace(" ", "_").lower())

#         # Skip if structured JSON already exists
#         if Path(f"/kaggle/working/{doc_id}_structured.json").exists():
#             print(f"✅ Already processed locally: {title}")
#             continue

#         print(f"⬇ Downloading: {title}")
#         file_path = DOWNLOAD_PATH / f"{sanitize_filename(title)}.docx"
#         file.GetContentFile(str(file_path), mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document')

#         loader = UnstructuredFileLoader(str(file_path))
#         pages = loader.load()
#         raw_text = pages[0].page_content
#         word_count = len(raw_text.split())

#         flat_chunks = []

#         for i, page in enumerate(pages):
#             chunks = chunk_text(page.page_content)
#             for j, chunk in enumerate(chunks):
#                 qa = generate_qa_pair(chunk)
#                 try:
#                     embedding = embedding_model.encode(chunk).tolist()
#                 except Exception as e:
#                     print(f"⚠️ Embedding error: {e}")
#                     embedding = None

#                 chunk_data = {
#                     "chunk_id": f"{doc_id}_chunk_{i}_{j}",
#                     "text": chunk,
#                     "embedding": embedding,
#                     "question": qa["question"],
#                     "answer": qa["answer"],
#                     "word_count": len(chunk.split()),
#                     "metadata": {
#                         "doc_id": doc_id,
#                         "title": title,
#                         "file_id": file_id,
#                         "modifiedDate": modified_time,
#                         "category": qa["category"]
#                     }
#                 }
#                 flat_chunks.append(chunk_data)

#         doc_entry = {
#             "doc_id": doc_id,
#             "metadata": {
#                 "title": title,
#                 "doc_id": doc_id,
#                 "file_id": file_id,
#                 "modifiedDate": modified_time,
#                 "word_count": word_count
#             },
#             "chunks": flat_chunks
#         }

#         # Save flat and structured JSONs in /kaggle/working
#         with open(f"/kaggle/working/{doc_id}_structured.json", "w") as f:
#             json.dump(doc_entry, f, indent=2)
#         with open(f"/kaggle/working/{doc_id}_flat.json", "w") as f:
#             json.dump(flat_chunks, f, indent=2)

#         try:
#             os.remove(file_path)
#         except Exception as e:
#             print(f"⚠️ Failed to delete file: {e}")

#     print("✅ Check complete.\n")

# if __name__ == "__main__":
#     check_for_new_files()


In [7]:
import os
import shutil
from pathlib import Path
import zipfile

WORKING_DIR = Path("/kaggle/working")
STRUCTURED_DIR = WORKING_DIR / "structured"
FLAT_DIR = WORKING_DIR / "flat"

# Create folders if they don't exist
STRUCTURED_DIR.mkdir(exist_ok=True)
FLAT_DIR.mkdir(exist_ok=True)

# Move JSON files into their respective folders
for file in WORKING_DIR.glob("*.json"):
    if file.name.endswith("_structured.json"):
        shutil.move(str(file), STRUCTURED_DIR / file.name)
    elif file.name.endswith("_flat.json"):
        shutil.move(str(file), FLAT_DIR / file.name)

# Function to zip a folder
def zip_folder(folder_path: Path, zip_path: Path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in folder_path.rglob('*'):
            zipf.write(file, arcname=file.relative_to(folder_path))

# Zip both folders
zip_folder(STRUCTURED_DIR, WORKING_DIR / "structured.zip")
zip_folder(FLAT_DIR, WORKING_DIR / "flat.zip")

print("✅ Folders zipped to:")
print(" - structured.zip")
print(" - flat.zip")


✅ Folders zipped to:
 - structured.zip
 - flat.zip
